In [9]:
import os
from dotenv import load_dotenv

load_dotenv() #load api 

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if OPENAI_API_KEY:
    print("api key found")
else:
    print("api key not found")

api key found


In [10]:

openai_model="gpt-4o-mini"


In [11]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(temperature=0.0,model=openai_model,api_key=OPENAI_API_KEY) # temperature for creativeness 

creative_llm = ChatOpenAI(temperature=0.9, model=openai_model,api_key=OPENAI_API_KEY)

In [18]:
from langchain_core.chat_history import BaseChatMessageHistory # abstract base class that defines the interface for chat message history implementations.
from langchain_core.chat_history import InMemoryChatMessageHistory # concrete implementation of BaseChatMessageHistory that stores messages in memory (RAM)
from langchain_core.runnables.history import RunnableWithMessageHistory # A wrapper that allows you to attach message history to a LangChain Runnable.
from langchain_core.messages import AIMessage # message class representing a message generated by the AI/LLM (Language Model) in a conversation.
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [19]:
result=llm.invoke([
    HumanMessage(content="Hi! I am cristanio"),
    AIMessage(content="Hello cristanio, How can i assist you today"),
    HumanMessage(content="what is my name")]
)

In [20]:
parser=StrOutputParser()
parser.invoke(result)

'Your name is Cristanio. How can I assist you today?'

In [21]:
result.content

'Your name is Cristanio. How can I assist you today?'

In [29]:
store={}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=InMemoryChatMessageHistory()
    return store[session_id]

In [27]:
config={"configurable": {"session_id":"first_chat"}}

In [30]:
model=RunnableWithMessageHistory(llm, get_session_history)
model.invoke([HumanMessage(content="Hi I am david billa")],config=config).content

'Hello, David Billa! How can I assist you today?'

In [32]:
model.invoke([HumanMessage(content="what is my name")],config=config).content

'Your name is David Billa. How can I help you today?'

In [33]:
store

{'first_chat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi I am david billa', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello, David Billa! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CArnsz0vtkis9kbzVvwxhQOk22CEU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--010c82c9-f51e-4609-9515-bfa643b70d2b-0', usage_metadata={'input_tokens': 13, 'output_tokens': 13, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessag

In [34]:
configg={"configurable":{"session_id":"second_chat"}}

In [36]:
model.invoke([HumanMessage(content="what is my name")],config=configg).content

"I'm sorry, but I don't have access to personal information about you unless you share it with me. How can I assist you today?"

In [38]:
store

{'first_chat': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi I am david billa', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello, David Billa! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 13, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CArnsz0vtkis9kbzVvwxhQOk22CEU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--010c82c9-f51e-4609-9515-bfa643b70d2b-0', usage_metadata={'input_tokens': 13, 'output_tokens': 13, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), HumanMessag

In [39]:
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate

prompt=ChatPromptTemplate.from_messages([
    ("system","you are a helpful assistant. answer all the question to the best of your knowledge"),
    MessagesPlaceholder(variable_name="messages"),
])

In [41]:
chain =prompt | llm

chain.invoke({"messages":["hi i am bob"]}).content

'Hello Bob! How can I assist you today?'

In [43]:
chain.invoke({"messages":[HumanMessage(content="what is my name")]}).content

"I'm sorry, but I don't have access to personal information about you, including your name. How can I assist you today?"

In [44]:
configgg={"configurable":{"session_id":"third_chat"}}

In [45]:
model=RunnableWithMessageHistory(chain, get_session_history)

respone=model.invoke([HumanMessage(content="Hi my name is jim")],config=configgg)

respone.content

'Hello, Jim! How can I assist you today?'

In [46]:
respone=model.invoke([HumanMessage(content="What is my name")],config=configgg)


In [47]:
respone.content

'Your name is Jim. How can I help you today, Jim?'

**trimming the conversation**

In [48]:
from langchain_core.messages import SystemMessage, trim_messages

In [49]:
trimmer=trim_messages(
    max_tokens=60,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

In [50]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [51]:
llm.get_num_tokens_from_messages(messages)

72

In [52]:
trimmer.invoke(messages)

[HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [53]:
llm.get_num_tokens_from_messages(trimmer.invoke(messages))

58

In [ ]:
prompt=ChatPromptTemplate.from_messages([
    ("system")
])

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

**Langchain with conversation Buffer memory**


In [2]:
from langchain.memory import ConversationBufferMemory
memory=ConversationBufferMemory()

/var/folders/j9/2xjjcspd5r3dz1m4ll_0c7m80000gn/T/ipykernel_6027/629970898.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()


In [3]:
memory.save_context({"input":"hi"},{"output":"what's up"})

In [5]:
memory.load_memory_variables({})

{'history': "Human: hi\nAI: what's up"}

In [7]:
memory2=ConversationBufferMemory(return_messages=True)
memory2.save_context({"input":"HI"},{"output":"what's up"})
memory2.load_memory_variables({})

{'history': [HumanMessage(content='HI', additional_kwargs={}, response_metadata={}),
  AIMessage(content="what's up", additional_kwargs={}, response_metadata={})]}

In [12]:
from langchain.chains import ConversationChain

conversation=ConversationChain(llm=llm,verbose=True,memory=ConversationBufferMemory())

/var/folders/j9/2xjjcspd5r3dz1m4ll_0c7m80000gn/T/ipykernel_6027/950388216.py:3: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation=ConversationChain(llm=llm,verbose=True,memory=ConversationBufferMemory())


In [13]:
conversation.predict(input="Hi there! my name is bob")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there! my name is bob
AI:

> Finished chain.


"Hello, Bob! It's great to meet you! I'm here to chat and help with anything you might need. How's your day going so far?"

In [14]:
conversation.predict(input="tell me 5 points about Databricks")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! my name is bob
AI: Hello, Bob! It's great to meet you! I'm here to chat and help with anything you might need. How's your day going so far?
Human: tell me 5 points about Databricks
AI:

> Finished chain.


'Sure, Bob! Here are five key points about Databricks:\n\n1. **Unified Analytics Platform**: Databricks is designed to unify data engineering, data science, and machine learning workflows. It provides a collaborative environment where teams can work together on data projects using a single platform.\n\n2. **Apache Spark Integration**: At its core, Databricks is built on Apache Spark, which is a powerful open-source engine for big data processing. This allows users to perform large-scale data processing and analytics efficiently.\n\n3. **Collaborative Notebooks**: Databricks offers interactive notebooks that support multiple languages, including Python, R, Scala, and SQL. These notebooks allow users to write code, visualize data, and share insights in a collaborative manner.\n\n4. **Delta Lake**: Databricks includes Delta Lake, an open-source storage layer that brings ACID transactions to big data workloads. This helps ensure data reliability and consistency, making it easier to manage 

In [15]:
conversation.memory.chat_memory.messages

[HumanMessage(content='Hi there! my name is bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello, Bob! It's great to meet you! I'm here to chat and help with anything you might need. How's your day going so far?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='tell me 5 points about Databricks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Sure, Bob! Here are five key points about Databricks:\n\n1. **Unified Analytics Platform**: Databricks is designed to unify data engineering, data science, and machine learning workflows. It provides a collaborative environment where teams can work together on data projects using a single platform.\n\n2. **Apache Spark Integration**: At its core, Databricks is built on Apache Spark, which is a powerful open-source engine for big data processing. This allows users to perform large-scale data processing and analytics efficiently.\n\n3. **Collaborative Notebooks**: Databricks offers intera

In [16]:
conversation.predict(input= "give me one point on hoe databricks better than synapse")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! my name is bob
AI: Hello, Bob! It's great to meet you! I'm here to chat and help with anything you might need. How's your day going so far?
Human: tell me 5 points about Databricks
AI: Sure, Bob! Here are five key points about Databricks:

1. **Unified Analytics Platform**: Databricks is designed to unify data engineering, data science, and machine learning workflows. It provides a collaborative environment where teams can work together on data projects using a single platform.

2. **Apache Spark Integration**: At its core, Databricks is built on Apache Spark, which is a powerful open-source engine for big data processing. This allows users to perform 

"One point where Databricks may have an advantage over Azure Synapse is its **collaborative environment for data science and machine learning**. Databricks provides interactive notebooks that support multiple programming languages and allow for real-time collaboration among team members. This feature is particularly beneficial for data scientists and analysts who need to work together on projects, share insights, and iterate quickly. While Azure Synapse also offers collaborative features, many users find Databricks' notebook interface to be more intuitive and user-friendly for data exploration and experimentation.\n\nIf you have more questions or need further comparisons, just let me know!"

In [17]:
conversation.memory.chat_memory.messages

[HumanMessage(content='Hi there! my name is bob', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello, Bob! It's great to meet you! I'm here to chat and help with anything you might need. How's your day going so far?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='tell me 5 points about Databricks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Sure, Bob! Here are five key points about Databricks:\n\n1. **Unified Analytics Platform**: Databricks is designed to unify data engineering, data science, and machine learning workflows. It provides a collaborative environment where teams can work together on data projects using a single platform.\n\n2. **Apache Spark Integration**: At its core, Databricks is built on Apache Spark, which is a powerful open-source engine for big data processing. This allows users to perform large-scale data processing and analytics efficiently.\n\n3. **Collaborative Notebooks**: Databricks offers intera

In [18]:
conversation.predict(input="what is my name")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there! my name is bob
AI: Hello, Bob! It's great to meet you! I'm here to chat and help with anything you might need. How's your day going so far?
Human: tell me 5 points about Databricks
AI: Sure, Bob! Here are five key points about Databricks:

1. **Unified Analytics Platform**: Databricks is designed to unify data engineering, data science, and machine learning workflows. It provides a collaborative environment where teams can work together on data projects using a single platform.

2. **Apache Spark Integration**: At its core, Databricks is built on Apache Spark, which is a powerful open-source engine for big data processing. This allows users to perform 

"Your name is Bob! If there's anything else you'd like to chat about or ask, feel free to let me know!"